In [122]:
import yfinance as yf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [123]:
# Load stock data using yfinance
spy = yf.Ticker("spy")
data = pd.DataFrame(spy.history(period="max", interval="1D"))
data = data[["Close", "Volume"]]
m, n = data.shape

In [129]:
# Calculate MACD and Signal Line
short_window = 12  # 12-period EMA
long_window = 26   # 26-period EMA
signal_window = 9  # 9-period EMA

# Calculate MACD and Signal Line
macd = data['Close'].ewm(span=short_window, adjust=False).mean() - data['Close'].ewm(span=long_window, adjust=False).mean()
signal = macd.ewm(span=signal_window, adjust=False).mean()

# Initialize tracking variables
isBullish_list = []
bullish_counter = 0
bearish_counter = 0

# Loop through the data to check for MACD crossover persistence
for i in range(1, len(macd)):
    if macd.iloc[i] > signal.iloc[i] and macd.iloc[i-1] > signal.iloc[i-1]:  # Golden Cross
        bullish_counter += 1
        bearish_counter = 0
        if bullish_counter >= 4:
            isBullish_list.append(1)  # Bullish
        else:
            isBullish_list.append(2)  # Neutral (before 2 consecutive days)
    elif macd.iloc[i] < signal.iloc[i] and macd.iloc[i-1] < signal.iloc[i-1]:  # Death Cross
        bearish_counter += 1
        bullish_counter = 0
        if bearish_counter >= 4:
            isBullish_list.append(-1)  # Bearish
        else:
            isBullish_list.append(2)  # Neutral (before 2 consecutive days)
    else:
        # Reset counters when there's no persistence of bullish/bearish signal
        bullish_counter = 0
        bearish_counter = 0
        isBullish_list.append(2)  # Neutral state

# Adjust length of isBullish_list for the first element
isBullish_list = [2] + isBullish_list  # Add 2 for the first row since we start from the second row

# Add the isBullish column to the dataset
data['MACD_Status'] = isBullish_list

# Check if the market is in an uptrend for atleast 5 days (TBI)
data['200_MA'] = EMA_200 = data['Close'].rolling(window=200).mean()# Preview the dataset
print(data)



                                Close    Volume  MACD_Status      200_MA
Date                                                                    
1993-01-29 00:00:00-05:00   24.684103   1003200            2         NaN
1993-02-01 00:00:00-05:00   24.859657    480500            2         NaN
1993-02-02 00:00:00-05:00   24.912352    201300            2         NaN
1993-02-03 00:00:00-05:00   25.175680    529400            2         NaN
1993-02-04 00:00:00-05:00   25.281010    531500            2         NaN
...                               ...       ...          ...         ...
2024-09-10 00:00:00-04:00  548.789978  36394600           -1  512.513175
2024-09-11 00:00:00-04:00  554.419983  75248600           -1  513.033644
2024-09-12 00:00:00-04:00  559.090027  51892700           -1  513.576078
2024-09-13 00:00:00-04:00  562.010010  39256500            2  514.137170
2024-09-16 00:00:00-04:00  562.840027  34378923            2  514.700185

[7964 rows x 4 columns]


In [125]:
#remove big tech earnings from data?



df.insert(0, 'index', list(range(0, m)))

X_train = df[["index", "Close", "Volume"]]

df["Target Column"] = df["Close"].shift(-1)
Y_train = df[["Target Column", "index"]]

print(X_train)
print(Y_train)

ValueError: cannot insert index, already exists